# Bootstrapping with Coulomb Potential

In this code, we demonstrate radial Coulomb potential with $\hbar = 1$ and $m = 1$. We begin with

\begin{align*}
    H &= \frac{P_r^2}{2m} + \frac{l(l+1)}{2r^2} + V(r)\\
    V(r) &= -\frac{e^2}{4\pi\epsilon_0r} = -\frac{k}{r}\\
    [r,P_r] &= i
\end{align*}

We see that the Hamiltonian is similar to 1-D Harmonic potential, with additional angular term $\frac{l(l+1)}{2r^2}$. So we just need to consider the additional commutation relation of the additional angular term. (Denote $P_r=P$ in the following)

\begin{align*}
    [H, r^{s}P] &= 0 = 
    -\frac{1}{2}s(s-1)\langle r^{s-2}P\rangle 
    -is\langle r^{s-1}P^2\rangle
    +i\langle r^{s}V^\prime(r)\rangle
    -il(l+1)\langle r^{s-3}\rangle\\
    
    [H, r^{s-1}] &= 0 =
    -\frac{1}{2}(s-1)(s-2)\langle r^{s-3}\rangle
    -i(s-1)\langle r^{s-2}P\rangle
\end{align*}

With the same trick and the same algebraic calcukation, we get the final recursion relation
\begin{equation*}
    0 = 2sE\langle r^{s-1}\rangle + \frac{1}{4}s(s-1)(s-2)\langle r^{s-3}\rangle 
    - \langle r^s V^\prime (r)\rangle - 2s\langle r^{s-1}V(r)\rangle -(s-1)l(l+1)\langle r^{s-3}\rangle
\end{equation*}

In [1]:
import os, time
import numpy as np
import sympy as sp
from bootstrap_sympy import sympy_solve_intervals, plot_energy_interval

import matplotlib.pyplot as plt
import seaborn as sns

## Recursion relation for 3-d Coulomb potential

Plug in $V(r)=-\frac{k}{r}$ into the recursion relation, we get

\begin{equation*}
    E\langle r^s\rangle = -\frac{2s+1}{2(s+1)}k\langle r^{s-1}\rangle
    + \frac{s}{2(s+1)}l(l+1)\langle r^{s-2}\rangle - \frac{s(s-1)}{8}\langle r^{s-2}\rangle
\end{equation*}

Since the energy eigenvalue is expected to be negative, and in order to transform it into a polynomial recursion in $E$, we set $E^\prime = -\frac{1}{E}$, and just denote $E^\prime$ as $E$ in the code below. With $\langle r^0\rangle=1$ by normalization, and $\langle r^1\rangle_{nlm}=\frac{a_0}{2}[3n^2-l(l+1)]$, where $a_0=\frac{4\pi\epsilon_0\hbar^2}{me^2}=\frac{\hbar^2}{mk}$.

In [2]:
class ColumbPotentialMatrix:
    def __init__(self, N, angular_momentum=True):
        self.k = sp.symbols('k') # constant of harmonic potential
        self.n = sp.symbols('n') # principal quantum number
        self.E = sp.symbols('E') # eigen-energy to be solved (Note the true energy is 1/E)
        self.l = sp.symbols('l') # angular momentum quantum number
        self.angular_momentum = angular_momentum
        
        self.rs_list = [] # list of r^s, the expectation value of position operator r to the s power
        for i in range(2*(N-1)+1):
            if i >= 2:
                self.rs_list.append(self.rs_resursion(s=i, rs_1=self.rs_list[i-1], rs_2=self.rs_list[i-2]))
            else:
                self.rs_list.append(self.rs_resursion(s=i))

    def rs_resursion(self, s, rs_1=None, rs_2=None):
        # find the <r^s> with recursion relation
        if s == 0:
            return 1
        elif s == 1:
            if self.angular_momentum:
                return (1/self.k)/2 * (3*self.n**2-self.l*(self.l+1))
            else:
                return (1/self.k)/2 * (3*self.n**2)
        else:
            if rs_1 == None or rs_2 == None:
                rs_1 = self.rs_resursion(s-1)
                rs_2 = self.rs_resursion(s-2)
            # factor out 1/8 to avoid overflow (sp.det will get error for small coefficient calculation)
            if self.angular_momentum:
                return (-self.E) * ( -4*(2*s+1)/(s+1)*self.k*rs_1 + 4*(s)/(s+1)*self.l*(self.l+1)*rs_2 - s*(s-1)*rs_2 ) / 8
            else:
                return (-self.E) * ( -4*(2*s+1)/(s+1)*self.k*rs_1 - s*(s-1)*rs_2 ) / 8

    def submatrix(self, L):
        # L*L matrix's [i,j] element = <r^(i+j)>
        return sp.Matrix([[self.rs_list[i+j] for j in range(L)] for i in range(L)])

In [5]:
cp_config = {
    'x_inf' : 0,
    'x_sup' : 10,
    'round' : 5,
    'plot_step' : 3,
    'threshold' : 1e-2,
    'initial_interval' : sp.Interval(-sp.oo, sp.oo),
}

cp_matrix = ColumbPotentialMatrix(N=cp_config['round'], angular_momentum=False)
cp_hyperparameters = {
    cp_matrix.k : 1,
    cp_matrix.n : 1,
    cp_matrix.l : 0,
}

for i in range(len(cp_matrix.rs_list)):
    print(sp.simplify(cp_matrix.rs_list[i]))
#     cp_matrix.rs_list[i] = sp.N(cp_matrix.rs_list[i], subs=cp_hyperparameters)
# energy_intervals, confirmed_intervals = sympy_solve_intervals(cp_matrix, cp_config, cp_hyperparameters, det_indep=False)

1
3*n**2/(2*k)
E*(1.25*n**2 + 0.25)
E*(0.875*E*k**2*(10.0*n**2 + 2) + 9*n**2)/(8*k)
E**2*(1.575*E*k**2*(10.0*n**2 + 2) + 46.2*n**2 + 6)/16
E**2*(0.90234375*E**2*k**4*n**2 + 0.18046875*E**2*k**4 + 5.38125*E*k**2*n**2 + 0.890625*E*k**2 + 2.8125*n**2)/k
E**3*(0.837890625*E**2*k**4*n**2 + 0.167578125*E**2*k**4 + 8.68828125*E*k**2*n**2 + 1.56529017857143*E*k**2 + 13.4397321428571*n**2 + 1.40625)
E**3*(0.7855224609375*E**3*k**6*n**2 + 0.1571044921875*E**3*k**6 + 12.882568359375*E**2*k**4*n**2 + 2.41492047991071*E**2*k**4 + 40.8513113839286*E*k**2*n**2 + 5.994140625*E*k**2 + 14.765625*n**2)/k
E**4*(0.74188232421875*E**3*k**6*n**2 + 0.14837646484375*E**3*k**6 + 18.0321044921875*E**2*k**4*n**2 + 3.45380510602679*E**2*k**4 + 99.3997628348214*E*k**2*n**2 + 16.6181640625*E*k**2 + 108.0234375*n**2 + 9.84375)
